In [12]:
import requests
from bs4 import BeautifulSoup
from nba_api.stats.endpoints import leaguedashteamstats, leaguedashteamshotlocations, commonallplayers
import pandas as pd
import numpy as np
import configparser
from sqlalchemy import create_engine
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [13]:
# Create an instance of the CommonAllPlayers endpoint
all_players_instance = commonallplayers.CommonAllPlayers()

# Retrieve the data
all_players = all_players_instance.get_data_frames()[0]  # Get the first DataFrame

# Extract relevant data
all_players = all_players[all_players['ROSTERSTATUS'] != 0]
all_players = all_players[['DISPLAY_FIRST_LAST', 'TEAM_CITY', 'TEAM_NAME']]
all_players['FIRST_NAME'] = all_players['DISPLAY_FIRST_LAST'].str.split().str[0]
all_players['LAST_NAME'] = all_players['DISPLAY_FIRST_LAST'].str.split().str[1]
all_players = all_players.drop(columns=['DISPLAY_FIRST_LAST'], axis =1)
all_players = all_players[['TEAM_NAME', 'FIRST_NAME', 'LAST_NAME']]
all_players

,TEAM_NAME,FIRST_NAME,LAST_NAME
10,Knicks,Precious,Achiuwa
22,Grizzlies,Steven,Adams
24,Heat,Bam,Adebayo
29,Jazz,Ochai,Agbaji
41,Grizzlies,Santi,Aldama
...,...,...,...
4881,Raptors,Thaddeus,Young
4883,Hawks,Trae,Young
4884,Jazz,Omer,Yurtseven
4887,Pelicans,Cody,Zeller


In [14]:
config = configparser.ConfigParser()
config.read('database.ini')

# Accessing database connection details
name = config['Database']['name']
host = config['Database']['host']
username = config['Database']['username']
password = config['Database']['password']


# Create the connection string
connection_str = f'postgresql+psycopg2://{username}:{password}@{host}:5432/{name}'

# Create the SQLAlchemy engine
engine = create_engine(connection_str)

In [16]:
### Get the TEAMS table from the database

result = engine.execute('select * from "Teams";')

# Fetch all rows from the query result
rows = result.fetchall()

# Get the column names from the query result
columns = result.keys()

# Create a DataFrame
data = pd.DataFrame(rows, columns=columns)

In [17]:
data

,id,city,name
0,1,Atlanta,Hawks
1,2,Boston,Celtics
2,3,Brooklyn,Nets
3,4,Charlotte,Hornets
4,5,Chicago,Bulls
5,6,Cleveland,Cavaliers
6,7,Dallas,Mavericks
7,8,Denver,Nuggets
8,9,Detroit,Pistons
9,10,Golden State,Warriors
